# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments           Customer  \
0  May 23 2022 10:25AM  237844        20  ALUR093906370  Alumier Labs Inc.   
1  May 23 2022 10:25AM  237844        20  ALUR229108759  Alumier Labs Inc.   
2  May 23 2022 10:25AM  237844        20  ALUR235029796  Alumier Labs Inc.   
3  May 23 2022 10:25AM  237844        20  ALUR251913830  Alumier Labs Inc.   
4  May 23 2022 10:25AM  237844        20  ALUR322301973  Alumier Labs Inc.   
5  May 23 2022 10:25AM  237844        20  ALUR342180041  Alumier Labs Inc.   
6  May 23 2022 10:25AM  237844        20  ALUR396168469  Alumier Labs Inc.   
7  May 23 2022 10:25AM  237844        20  ALUR624272773  Alumier Labs Inc.   
8  May 23 2022 10:25AM  237844        20  ALUR634082265  Alumier Labs Inc.   
9  May 23 2022 10:25AM  237844        20  ALUR649803461  Alumier Labs Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
34  237840       Released          1
35  237841       Released         58
36  237843       Released          1
37  237844       Released         58
38  237847       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
237840                NaN        NaN       1.0
237841                NaN        NaN      58.0
237843                NaN        NaN       1.0
237844                NaN        NaN      58.0
237847                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237781                0.0        0.0      51.0
237787                0.0        1.0       0.0
237789                1.0        6.0       0.0
237792                0.0        3.0      15.0
237794                0.0        0.0      26.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237781                  0          0        51
237787                  0          1         0
237789                  1          6         0
237792                  0          3        15
237794                  0          0        26

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               237781          0          0        51
1               237787          0          1         0
2               237789          1          6         0
3               237792          0          3        15
4               237794          0          0        26

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               237781                           51
1               237787                   1         
2               237789         1         6         
3               237792                   3       15
4               237794                           26

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0    May 23 2022 10:25AM  237844        20   
58   May 23 2022 10:24AM  237847        19   
59   May 23 2022 10:13AM  237841        10   
117  May 23 2022 10:09AM  237843        10   
118  May 23 2022 10:08AM  237836        10   
169  May 23 2022  9:59AM  237840        12   
170  May 23 2022  9:58AM  237839        10   
173  May 23 2022  9:56AM  237837        16   
174  May 23 2022  9:53AM  237830        10   
237  May 23 2022  9:48AM  237832        19   

                                 Customer  
0                       Alumier Labs Inc.  
58                GUSA Granules USA, Inc.  
59                      ISDIN Corporation  
117                     ISDIN Corporation  
118                     ISDIN Corporation  
169                        Organimed, LLC  
170                     ISDIN Corporation  
173                  Emersa Waterbox, LLC  
174                     ISDIN Corporation  
237  Bi-Coastal Pharma International, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  May 23 2022 10:25AM  237844        20   
1  May 23 2022 10:24AM  237847        19   
2  May 23 2022 10:13AM  237841        10   
3  May 23 2022 10:09AM  237843        10   
4  May 23 2022 10:08AM  237836        10   
5  May 23 2022  9:59AM  237840        12   
6  May 23 2022  9:58AM  237839        10   
7  May 23 2022  9:56AM  237837        16   
8  May 23 2022  9:53AM  237830        10   
9  May 23 2022  9:48AM  237832        19   

                               Customer Completed Executing Released  
0                     Alumier Labs Inc.                           58  
1               GUSA Granules USA, Inc.                            1  
2                     ISDIN Corporation                           58  
3                     ISDIN Corporation                            1  
4                     ISDIN Corporation                           51  
5                        Organimed, LLC                            1  
6                     ISDIN Corporation                            3  
7                  Emersa Waterbox, LLC                            1  
8                     ISDIN Corporation                           63  
9  Bi-Coastal Pharma International, LLC                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  May 23 2022 10:25AM  237844        20   
1  May 23 2022 10:24AM  237847        19   
2  May 23 2022 10:13AM  237841        10   
3  May 23 2022 10:09AM  237843        10   
4  May 23 2022 10:08AM  237836        10   
5  May 23 2022  9:59AM  237840        12   
6  May 23 2022  9:58AM  237839        10   
7  May 23 2022  9:56AM  237837        16   
8  May 23 2022  9:53AM  237830        10   
9  May 23 2022  9:48AM  237832        19   

                               Customer Released Executing Completed  
0                     Alumier Labs Inc.       58                      
1               GUSA Granules USA, Inc.        1                      
2                     ISDIN Corporation       58                      
3                     ISDIN Corporation        1                      
4                     ISDIN Corporation       51                      
5                        Organimed, LLC        1                      
6                     ISDIN Corporation        3                      
7                  Emersa Waterbox, LLC        1                      
8                     ISDIN Corporation       63                      
9  Bi-Coastal Pharma International, LLC        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                 Customer Released  \
0  May 23 2022 10:25AM  237844        20        Alumier Labs Inc.       58   
1  May 23 2022 10:24AM  237847        19  GUSA Granules USA, Inc.        1   
2  May 23 2022 10:13AM  237841        10        ISDIN Corporation       58   
3  May 23 2022 10:09AM  237843        10        ISDIN Corporation        1   
4  May 23 2022 10:08AM  237836        10        ISDIN Corporation       51   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                 Customer  Released  \
0  May 23 2022 10:25AM  237844        20        Alumier Labs Inc.      58.0   
1  May 23 2022 10:24AM  237847        19  GUSA Granules USA, Inc.       1.0   
2  May 23 2022 10:13AM  237841        10        ISDIN Corporation      58.0   
3  May 23 2022 10:09AM  237843        10        ISDIN Corporation       1.0   
4  May 23 2022 10:08AM  237836        10        ISDIN Corporation      51.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                 Customer  Released  \
0  May 23 2022 10:25AM  237844        20        Alumier Labs Inc.      58.0   
1  May 23 2022 10:24AM  237847        19  GUSA Granules USA, Inc.       1.0   
2  May 23 2022 10:13AM  237841        10        ISDIN Corporation      58.0   
3  May 23 2022 10:09AM  237843        10        ISDIN Corporation       1.0   
4  May 23 2022 10:08AM  237836        10        ISDIN Corporation      51.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3805119     419.0       15.0        0.0
12          237840       1.0        0.0        0.0
15         1426797      77.0        3.0        0.0
16          237837       1.0        0.0        0.0
18         1426868       6.0        0.0        0.0
19          713468       2.0        6.0        1.0
20          237844      58.0        0.0        0.0
22          237787       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3805119     419.0       15.0        0.0
1        12   237840       1.0        0.0        0.0
2        15  1426797      77.0        3.0        0.0
3        16   237837       1.0        0.0        0.0
4        18  1426868       6.0        0.0        0.0
5        19   713468       2.0        6.0        1.0
6        20   237844      58.0        0.0        0.0
7        22   237787       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     419.0       15.0        0.0
1        12       1.0        0.0        0.0
2        15      77.0        3.0        0.0
3        16       1.0        0.0        0.0
4        18       6.0        0.0        0.0
5        19       2.0        6.0        1.0
6        20      58.0        0.0        0.0
7        22       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  419.0
1        12  Released    1.0
2        15  Released   77.0
3        16  Released    1.0
4        18  Released    6.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   16   18   19    20   22
Status                                               
Completed    0.0  0.0   0.0  0.0  0.0  1.0   0.0  0.0
Executing   15.0  0.0   3.0  0.0  0.0  6.0   0.0  1.0
Released   419.0  1.0  77.0  1.0  6.0  2.0  58.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   16   18   19    20   22
0          Completed    0.0  0.0   0.0  0.0  0.0  1.0   0.0  0.0
1          Executing   15.0  0.0   3.0  0.0  0.0  6.0   0.0  1.0
2           Released  419.0  1.0  77.0  1.0  6.0  2.0  58.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   16   18   19    20   22
0  Completed    0.0  0.0   0.0  0.0  0.0  1.0   0.0  0.0
1  Executing   15.0  0.0   3.0  0.0  0.0  6.0   0.0  1.0
2   Released  419.0  1.0  77.0  1.0  6.0  2.0  58.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()